In [1]:
import numpy as np
import matplotlib.pyplot as plt
from module import generate_target_function, generate_data, pla, calculate_error, linear_target_function, linear_regression

## Linear Regression
Do 1000 executions of:
1. Choose a target function.
2. Generate data from the distribution $(X \in [1, -1] \times [1, -1])$ and classify it with the target function.
3. Run the Linear Regression algorithm
4. Evaluate on a set of 1000 test points.

In [2]:
# Experimento
def experiment_lr(num_runs, num_points_train, num_points_test):
    ein_list = []
    eout_list = []

    for _ in range(num_runs):
        a, b, c = generate_target_function()
        lg_tg = linear_target_function(a, b, c)
        X_train, y_train = generate_data(num_points_train, lg_tg)
        w = linear_regression(X_train, y_train)
        ein = calculate_error(X_train, y_train, w)
        ein_list.append(ein)

        # Gerar novos pontos de teste
        X_test, y_test = generate_data(num_points_test, lg_tg)
        eout = calculate_error(X_test, y_test, w)
        eout_list.append(eout)

    return np.mean(ein_list), np.std(ein_list), np.mean(eout_list), np.std(eout_list)



#### Consider N = 100. Use Linear Regression to find g and calculate Ein, the fraction of points within the sample that were classified incorrectly Repeat the experiment 1000 times. Which of the values below is closest to the average Ein?

In [3]:
results_lg = experiment_lr(1000, 100, 1000)

In [ ]:
print('Ein mean: ', results_lg[0])
print('Ein std: ', results_lg[1])

Ein mean:  0.03851
Ein std:  0.030493932183304928


#### Now, generate 1000 new points and use them to estimate the Eout of the g's you found in item. Again, perform 1000 runs. Which of the values below is closest to the average Eout?

In [ ]:
print('Eout mean: ', results_lg[2])
print('Eout std: ', results_lg[3])

Eout mean:  0.048468000000000004
Eout std:  0.03204788567128883


####  Now, consider N = 10. After finding the weights using Linear Regression, use them as an initial vector for the Perceptron Learning Algorithm (PLA). Run the PLA until it converges on a final vector of weights that perfectly separates the within-sample points. Among options below, which is closest to the average number of iterations (over 1000 runs) that the PLA takes to converge?

In [ ]:
def experiment_rg_pla(num_runs, num_points_train):
    iterations_list = []

    for _ in range(num_runs):
        a, b, c = generate_target_function()
        lg_tg = linear_target_function(a, b, c)
        X_train, y_train = generate_data(num_points_train, lg_tg)
        w = linear_regression(X_train, y_train)
        w, iterations = pla(X_train, y_train, w)
        iterations_list.append(iterations)

    return np.mean(iterations_list), np.std(iterations_list)

In [ ]:
# Parâmetros
num_runs = 1000
num_points_train = 10

# Executar o experimento
mean_iterations, std_iterations = experiment_rg_pla(num_runs, num_points_train)
print(f"Iterations mean: {mean_iterations}")
print(f"Iterations std: {std_iterations}")

Iterations mean: 3.465
Iterations std: 11.045215027331972
